In [1]:
"""""""""""""""""""""""""""""""""""""""
Модуль создания и обучения нейронной сети для распознавания рукописных цифр с использованием метода градиентного спуска.
Группа: АСУб-20-1
ФИО: Устюжанин Вадим Максимович
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
import random # библиотека функций для генерации случайных значений
import numpy as np # библиотека функций для работы с матрицами

""" ---Раздел описаний--- """
""" --Описание класса Network--"""
class Network(object): # используется для описания нейронной сети
    def __init__(self, sizes): # конструктор класса, self – указатель на объект класса, sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes) # задаем количество слоев нейронной сети
        self.sizes = sizes # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] # задаем случайные начальные веса связей
            
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD( # Стохастический градиентный спуск
              self # указатель на объект класса
            , training_data # обучающая выборка
            , epochs # количество эпох обучения
            , mini_batch_size # размер подвыборки
            , eta # скорость обучения
            , test_data # тестирующая выборка
            ):
        test_data = list(test_data) # создаем список объектов тестирующей выборки
        n_test = len(test_data) # вычисляем длину тестирующей выборки
        training_data = list(training_data) # создаем список объектов обучающей выборки
        n = len(training_data) # вычисляем размер обучающей выборки
        for j in range(epochs): # цикл по эпохам
            random.shuffle(training_data) # перемешиваем элементы обучающей выборки
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
            for mini_batch in mini_batches: # цикл по подвыборкам
                self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
            print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)) # смотрим прогресс в обучении    
            with open('biases.pkl', 'wb') as f:
                pickle.dump(self.biases, f)
            with open('weights.pkl', 'wb') as f:
                pickle.dump(self.weights, f)
            
    def update_mini_batch( # Шаг градиентного спуска
                          self # указатель на объект класса
                        , mini_batch # подвыборка
                        , eta # скорость обучения
                        ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети
        
    def backprop( # Алгоритм обратного распространения
                  self # указатель на объект класса
                , x # вектор входных сигналов
                , y # ожидаемый вектор выходных сигналов
                ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

        # определение переменных
        activation = x # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x] # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = [] # список активационных потенциалов по всем слоям (первоначально пуст)

        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b # считаем активационные потенциалы текущего слоя
            zs.append(z) # добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z) # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation) # добавляем элемент (выходные сигналы слоя) в конец списка

        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta # градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # градиент dC/dw для слоя L (BP4)
        for l in range(2, self.num_layers):
            z = zs[-l] # активационные потенциалы l-го слоя (двигаемся "Положение EcoRobotics.docx"списку справа налево)
            sp = sigmoid_prime(z) # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta # градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())# градиент dC/dw для l-го слоя (BP4)
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data): # Оценка прогресса в обучении
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
        return (output_activations-y)
    
        """ --Конец описания класса Network--"""
""" --- Конец раздела описаний--- """

""" ---Тело программы--- """
net = Network([2, 3, 1]) # создаем нейронную сеть из трех слоев
""" ---Конец тела программы--- """
""" Вывод результата на экран: """
print('Сеть net:')
print('Количетво слоев:', net.num_layers)
for i in range(net.num_layers):
    print('Количество нейронов в слое', i,':',net.sizes[i])
for i in range(net.num_layers-1):
    print('W_',i+1,':')
    print(np.round(net.weights[i],2))
    print('b_',i+1,':')
    print(np.round(net.biases[i],2))

Сеть net:
Количетво слоев: 3
Количество нейронов в слое 0 : 2
Количество нейронов в слое 1 : 3
Количество нейронов в слое 2 : 1
W_ 1 :
[[-0.21 -3.21]
 [ 1.6   0.72]
 [-1.    0.72]]
b_ 1 :
[[0.09]
 [0.08]
 [0.27]]
W_ 2 :
[[ 0.19  0.42 -0.5 ]]
b_ 2 :
[[1.83]]


In [2]:
def sigmoid(z): # определение сигмоидальной функции активации
        return 1.0/(1.0+np.exp(-z))
    
def sigmoid_prime(z):# Производная сигмоидальной функции
        return sigmoid(z)*(1-sigmoid(z))

In [3]:
import gzip # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np # библиотека для работы с матрицами

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb') # открываем сжатый файл gzip в двоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # загружам таблицы из файла
    f.close() # закрываем файл
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data() # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]] # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results) # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1]) # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1]) # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1
    return e

training_data, validation_data,test_data = load_data_wrapper()

net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9033 / 10000
Epoch 1: 9232 / 10000
Epoch 2: 9332 / 10000
Epoch 3: 9359 / 10000
Epoch 4: 9345 / 10000
Epoch 5: 9346 / 10000
Epoch 6: 9400 / 10000
Epoch 7: 9410 / 10000
Epoch 8: 9448 / 10000
Epoch 9: 9447 / 10000
Epoch 10: 9457 / 10000
Epoch 11: 9437 / 10000
Epoch 12: 9441 / 10000
Epoch 13: 9463 / 10000
Epoch 14: 9456 / 10000
Epoch 15: 9464 / 10000
Epoch 16: 9486 / 10000
Epoch 17: 9480 / 10000
Epoch 18: 9478 / 10000
Epoch 19: 9465 / 10000
Epoch 20: 9475 / 10000
Epoch 21: 9457 / 10000
Epoch 22: 9468 / 10000
Epoch 23: 9482 / 10000
Epoch 24: 9472 / 10000
Epoch 25: 9482 / 10000
Epoch 26: 9508 / 10000
Epoch 27: 9493 / 10000
Epoch 28: 9512 / 10000
Epoch 29: 9487 / 10000


In [4]:
import numpy as np # библиотека функций для работы с матрицами
import pickle

def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

class Network(object): # используется для описания нейронной сети
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def __init__(self, sizes): # конструктор класса
    # self – указатель на объект класса
    # sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes) # задаем количество слоев нейронной сети
        self.sizes = sizes # задаем список размеров слоев нейронной сети

        with open('biases.pkl', 'rb') as f: #открытие файла со смещениями
            self.biases = pickle.load(f)
        with open('weights.pkl', 'rb') as f: #открытие файла с весами
            self.weights = pickle.load(f)

# Определение двузначных чисел

Создание резервных файлов copy, left и right 

copy - используется как буфер для разбиения числа на два

left и right это разбитое на два десятичное число

left - хранит в себе левую часть числа
right - хранит в себе правую часть числа

In [17]:
from PIL import Image

new_image = Image.new("RGB", (28, 28), color = 'black')
new_image = new_image.convert('P')

new_image.save("copy.bmp")
new_image.save("left.bmp")
new_image.save("second.bmp")

In [60]:
def getImage(number): #функция для определения изображения
    image = Image.open("copy.bmp") #открываем копию, чтобы не сломать исходный файл
    image = image.convert("RGB")
    
    width, height = image.size
    white_pixels_count = 0 #счётчик белых пикселей
    
    x1, y1, x2, y2 = 0, height, width, 0

    for x in range(width):
        for y in range(height):
            if image.getpixel((x, y)) == (255, 255, 255):
                white_pixels_count +=1
                if y > y2: y2 = y #поиск нижней границы y
                if y < y1: y1 = y #поиск верхней границы y
                if x1 == 0: x1 = x - 1 #поиск левой границы y
        #поиск правой границы x и выход и цикла
        if white_pixels_count == 0 and x1 != 0:
            x2 = x
            break
        white_pixels_count = 0
        
    #обрезка левой цифры
    num_1 = image.crop((x1, y1, x2, y2))
    num_1.save(number)
 
    #поиск правой цифры
    image = image.crop((x2, 0, width, height))
    image.save("copy.bmp")
    width, height = num_1.size
    
    #определение области изображения второй цифры
    if width > height:
        length = round(width * 1.25)
    else:
        length = round(height * 1.25)
      
    new_image = Image.new("RGB", (length, length), color = 'black')
    #определение координат правой цифры
    x = int((length - width)/2)
    y = int((length - height)/2)
    
    new_image.paste(num_1, (x, y, x + width, y + height)) #вставка цифры в num_1
    new_image = new_image.resize((28, 28))
    new_image = new_image.convert('P')
    
    new_image.save(number)
    
def determine(name): #функция с определением чисел через нейросеть
    if __name__ == "__main__":
        
        img = Image.open(name)
        img.save("copy.bmp")

        #получаем картинки с цифрами
        getImage("left.bmp")
        getImage("right.bmp")

        #определяем левую часть
        im=Image.open("left.bmp")
        testArray=np.ndarray(shape=(784, 1), dtype=float, order='F')

        for i2 in range(28):
            for i1 in range(28):
                if (im.getpixel((i1, i2))) / 255 > 0:
                    #вынужден был применять костыль, т.к. 1 в значении пикселей нейросеть неадекватно воспринимает
                    testArray[i1 + i2 * 28, 0] = (im.getpixel((i1, i2))) / 255 - 0.001
                else:
                    testArray[i1 + i2 * 28,0] = (im.getpixel((i1, i2))) / 255

        net = Network([784, 30, 10]) #создаем нейросеть с обученными весами и смещениями
        print(net.feedforward(testArray)) #распознаем изображение с помощью нейросети
        
        #определяем правую часть
        im=Image.open("right.bmp")
        testArray=np.ndarray(shape=(784, 1), dtype=float, order='F')

        for i2 in range(28):
            for i1 in range(28):
                if (im.getpixel((i1, i2))) / 255 > 0:
                    #вынужден был применять костыль, т.к. 1 в значении пикселей нейросеть неадекватно воспринимает
                    testArray[i1 + i2 * 28, 0] = (im.getpixel((i1, i2))) / 255 - 0.001
                else:
                    testArray[i1 + i2 * 28,0] = (im.getpixel((i1, i2))) / 255

        net = Network([784, 30, 10]) #создаем нейросеть с обученными весами и смещениями
        print(net.feedforward(testArray)) #распознаем изображение с помощью нейросети
        
        #выводим картинку
        im

In [61]:
determine("18.bmp")
im=Image.open("18.bmp")
im

[[1.31062004e-12]
 [2.67565943e-01]
 [8.42927692e-10]
 [2.90119728e-06]
 [1.15953815e-08]
 [6.38889729e-08]
 [2.68804322e-09]
 [4.43043488e-01]
 [4.16581851e-05]
 [4.26065279e-03]]
[[1.06152193e-05]
 [1.47637064e-03]
 [9.97746811e-01]
 [8.44471621e-05]
 [3.12281828e-11]
 [2.52549524e-07]
 [1.31750187e-03]
 [3.03851543e-06]
 [2.18127649e-07]
 [1.86858601e-06]]


In [62]:
determine("29.bmp")
im=Image.open("29.bmp")
im

[[1.01375614e-06]
 [1.26705421e-04]
 [4.89886699e-01]
 [3.60941906e-05]
 [1.93901749e-07]
 [1.44705456e-05]
 [1.10868253e-05]
 [1.31682305e-03]
 [1.18056881e-07]
 [1.20908552e-09]]
[[3.92140350e-05]
 [1.44715561e-05]
 [1.24906099e-05]
 [1.98812759e-04]
 [1.82282484e-10]
 [1.97558146e-02]
 [8.99503805e-10]
 [6.12719043e-03]
 [2.28728473e-07]
 [6.82836872e-05]]


In [63]:
determine("30.bmp")
im=Image.open("30.bmp")
im

[[8.07548767e-08]
 [2.07607153e-10]
 [1.70030867e-10]
 [2.40704720e-01]
 [4.81309070e-04]
 [5.97835585e-05]
 [4.81209545e-04]
 [1.92255727e-06]
 [6.43273405e-05]
 [2.36451035e-08]]
[[8.38015536e-04]
 [1.26678375e-04]
 [3.78802925e-07]
 [1.20824534e-05]
 [5.12848892e-10]
 [1.13186732e-04]
 [4.29727952e-04]
 [1.12227220e-05]
 [8.25222411e-03]
 [3.15874604e-10]]


In [64]:
determine("47.bmp")
im=Image.open("47.bmp")
im

[[1.91564361e-09]
 [1.47243375e-10]
 [8.42062780e-06]
 [6.80640593e-04]
 [3.92564703e-01]
 [1.26182341e-04]
 [1.74026044e-10]
 [6.64777336e-06]
 [3.79880400e-03]
 [2.26983442e-01]]
[[7.93858809e-13]
 [1.19454048e-04]
 [5.56333811e-09]
 [1.43141463e-09]
 [2.81734155e-08]
 [1.78308183e-05]
 [6.21608662e-02]
 [1.98359304e-04]
 [2.21198587e-10]
 [1.37638465e-03]]


In [65]:
determine("56.bmp")
im=Image.open("56.bmp")
im

[[1.51166819e-05]
 [2.00431607e-09]
 [4.67866073e-08]
 [1.90372923e-05]
 [1.05263293e-06]
 [9.86887215e-01]
 [2.38019790e-07]
 [1.35361940e-08]
 [7.22140043e-03]
 [2.28676114e-03]]
[[1.46103023e-03]
 [1.70331962e-09]
 [1.15045689e-03]
 [8.87628088e-06]
 [6.61054440e-09]
 [1.04105055e-02]
 [9.94136634e-01]
 [1.20997650e-10]
 [3.94600529e-06]
 [4.31798184e-10]]
